# ** Описание **

In [1]:
from __future__ import division
import scipy
import binascii
from sklearn.metrics import f1_score
import numpy as np
import base64
from sklearn.linear_model import LogisticRegression
import csv
from nltk.tokenize import RegexpTokenizer
import gzip
import pymorphy2
import zlib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import importlib
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import xgboost
import zipfile
import gzip
import string
import pickle
import re
import sys
from io import BytesIO
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from collections import namedtuple
from scipy import sparse
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
%matplotlib inline
from bs4 import BeautifulSoup
from string import punctuation
from bs4.element import Comment
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.neural_network import MLPClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vkrin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier

In [3]:
TRACE_NUM = 100
import logging
importlib.reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

### Извлечение текста

In [4]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]'] or isinstance(element, Comment):
        return False
    else:
        return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    
    links = []
    links_t=''
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
        links_t+=(' '+link.get_text())
    
    return ' '.join(visible_texts), links, links_t

def tokenize_me(file_text):
    tokens = nltk.word_tokenize(file_text)
    tokens = [morph.parse(i.lower())[0].normal_form for i in tokens if ( i not in string.punctuation )]
 
    stop_words = stopwords.words('russian')+stopwords.words('english')
    stop_words.extend(map(lambda x: x, 
                          ['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на']))
    tokens = [i for i in tokens if ( i not in stop_words )]     
    return ' '.join(tokens)

In [5]:
def html2text_bs(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

def html2text_title(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения title
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    s=soup.find_all('title')
    if s:
        return tokenize_me(s[0].get_text())
    return ''

def html2text_ss(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения ссылок и анкоров
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    links=[]
    texts=''
    for link in soup.find_all('a', href=True):
        links.append(link['href'])
        for text in link.get_text().split():
            texts=texts+' '+text
    return ' '.join(links),' '.join(tokenize_me(texts))

def html2text_bs_visible(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста, который видим пользователю
    """
    soup = BeautifulSoup(raw_html, "html.parser")    
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    return ' '.join(tokenize_me(soup.get_text()))

def html2text_boilerpipe(raw_html):
    import boilerpipe
    """
    еще одна библиотека очень хорошо извлекающая именно видимый пользователю текст,
    но она завязана на java
    """
    pass

#### Рассчет финальных метрик

In [6]:
def safe_divide(a, b):
    if a == 0: return 0.0
    elif b == 0: return 0.0
    else: return a/b

def calculate_metrics(predictions, threshold):    
    """
    Функция подсчета метрик
    Параметры
    predictions - ранки по документам
    threshold  - порог для метрик
    """
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for (url_id, mark, url, prediction) in predictions:        
        mark_predict = prediction > threshold

        if mark_predict:                     
            if mark_predict == mark: true_positive += 1
            else: false_positive += 1                    
        else:                     
            if  mark_predict == mark: true_negative += 1
            else: false_negative += 1

    class_prec  = safe_divide(true_positive, true_positive + false_positive)
    class_recall = safe_divide(true_positive, true_positive + false_negative)
        
    class_F1 = safe_divide(2 * class_prec * class_recall, class_prec + class_recall)
    
    
    not_class_prec = safe_divide(true_negative, true_negative + false_negative)
    not_class_recall = safe_divide(true_negative, true_negative + false_positive)
    
    not_class_F1 = safe_divide(2 * not_class_prec * not_class_recall, not_class_prec + not_class_recall)
    
    return ( (class_prec, class_recall, class_F1), (not_class_prec, not_class_recall, not_class_F1) )

def arange(start, stop, step):
    cur_value = start
    while True:
        if cur_value > stop: break
        yield cur_value
        cur_value += step

def plot_results(docs, min_threshold=-1, max_threshold=1, step=0.1, trace=False):
    x = []
    y_p = []
    y_n = []
    docs_predictions = classifier.predict_all(docs)
    for threshold in arange(min_threshold, max_threshold, step):
        r = calculate_metrics(docs_predictions, threshold)
        x.append(threshold)
        y_p.append(r[0])
        y_n.append(r[1])        
        if trace: 
            print ('threshold %s',  threshold)
            print ('\tclass_prec %s, class_recall %s, class_F1 %s',  r[0])
            print ('\tnot_class_prec %s, not_class_recall %s, not_class_F1 %s',  r[1])
            print ('\t\tMacroF1Mesure %s',  ((r[0][2] + r[1][2])/2))
    plot_stats(x, y_p, "Class Result")
    plot_stats(x, y_n, "Not class Result")    


def plot_stats(x, y, title):
    plt.figure(figsize=(10, 5))

    prec, = plt.plot( x, 
                     [k[0] for k in y], "r", label='Precision', 
                     linewidth=1)
    accur, = plt.plot( x, 
                      [k[1] for k in y], "b", label='Recall',
                      linewidth=1)
    f1, =    plt.plot( x, 
                      [k[2] for k in y], "g", label='F1',
                      linewidth=1)
    plt.grid(True)
    plt.legend(handles=[prec, accur, f1])
    plt.title(title)
    plt.show()

In [7]:
def calc_features(url, html_data,title=False):
    text, links, ankors = text_from_html(html_data)
    text=tokenize_me(text)
    ankors=tokenize_me(ankors)
    all_text=tokenize_me(text) + ' '.join(links)+' '+tokenize_me(ankors)
    words = text.split()
    words_num = len(words)
    ankors_num=len(ankors.split())
    urls_num=len(links)
    if words_num>0:
        uniq=len(set(words))/words_num
    else:
        uniq=0
    len_url=1/len(url)
    compression_level = len(gzip.compress(html_data))/len(html_data)
    return [len(words), compression_level, all_text, uniq, urls_num,len_url, ankors_num, ankors]

In [10]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url'])#, 'features'])

def load_csv(input_file_name, calc_features_f):    
    """
    Загружаем данные и извлекаем на лету признаки
    Сам контент не сохраняется, чтобы уменьшить потребление памяти - чтобы
    можно было запускать даже на ноутбуках в классе
    """
    
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name,encoding='utf8')  as input_file:            
        headers = input_file.readline()
        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.decode('utf8').strip().split('\t')
            url_id = int(parts[0])                                        
            mark = bool(int(parts[1]))                    
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)
            features = calc_features_f(url, html_data)            
            yield DocItem(url_id, mark, url, features)            
                
        trace(i, 1)  

In [ ]:
%%time

TRAIN_DATA_FILE  = 'kaggle_train_data_tab.csv.gz'
TEST_DATA_FILE  = 'kaggle_test_data_tab.csv.gz'

train_docs =list(load_csv(TRAIN_DATA_FILE, calc_features))
test_docs = list(load_csv(TEST_DATA_FILE, calc_features))

In [ ]:
pickle.dump(train_docs, open('train.p', 'wb'))
pickle.dump(test_docs, open('test.p', 'wb'))

In [ ]:
input_train = open('train.p', 'rb')
input_test = open('test.p', 'rb')
train_docs=pickle.load(input_train)
test_docs=pickle.load(input_test)
input_test.close()
input_train.close()

In [ ]:
vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.001, use_idf=True, ngram_range=(1,3))

In [ ]:
X_train=vectorizer.fit_transform(generator(train_docss=train_docs))
X_test=vectorizer.transform(generator(train_docss=test_docs))

In [ ]:
N=3000
voc=set()
idx = np.ravel(X_train.sum(axis=0).argsort(axis=1))[::-1][:N]
top_words = np.array(vectorizer.get_feature_names())[idx].tolist()
for word in top_words:
    voc.add(word)
print(voc)

In [ ]:
f1=[]
for doc in train_docs:
    f1.append(np.array([np.float64(doc[3][0]),np.float64(doc[3][1]),np.float64(doc[3][3]),np.float64(doc[3][4]),np.float64(doc[3][5]),np.float64(doc[3][6])]))
f1=np.array(f1)
f2=[]
for doc in test_docs:
    f2.append(np.array([np.float64(doc[3][0]),np.float64(doc[3][1]),np.float64(doc[3][3]),np.float64(doc[3][4]),np.float64(doc[3][5]),np.float64(doc[3][6])]))
f2=np.array(f2)

In [ ]:
y_train=[]
for doc in train_docs:
    y_train.append(float(doc[1]))
y_train=np.array(y_train)

In [ ]:
x_vs=sparse.csc_matrix(f1).tocsr()
X_train_=sparse.hstack([X_train, x_vs]).tocsr()
x_vs=sparse.csc_matrix(f2).tocsr()
X_test_=sparse.hstack([X_test, x_vs]).tocsr()

In [ ]:
np.save('y_train.npy', y_train)
scipy.sparse.save_npz('train.npz', X_train_)
scipy.sparse.save_npz('test.npz', X_test_)

In [20]:
X_train_=scipy.sparse.load_npz('train.npz')
X_test_=scipy.sparse.load_npz('test.npz')
y_train=np.load('y_train.npy')

In [13]:
x1, x2, y1, y2=train_test_split(X_train_, y_train,  test_size=0.1, random_state=42)

In [14]:
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=1)
clf2 = MultinomialNB()
clf3 = SGDClassifier(alpha=0.0001, average=False,
                     class_weight=None,
                     early_stopping=False, epsilon=0.1,
                     eta0=0.0, fit_intercept=True,
                     l1_ratio=0.15,
                     learning_rate='optimal',
                     max_iter=1000,loss='log',
                     n_iter_no_change=5, n_jobs=None,
                     penalty='l2', power_t=0.5,
                     random_state=None, shuffle=True,
                     tol=0.001, validation_fraction=0.1)
clf4= xgboost.XGBClassifier(max_depth=6,learning_rate=0.3,n_estimators=1000,scale_pos_weight=1.5, n_jobs=6,probability=True)
clf6=SVC(C=1.0, cache_size=200, class_weight=None,
                                  coef0=0.0, decision_function_shape='ovr',
                                  degree=3, gamma='auto', kernel='rbf',
                                  max_iter=-1,
                                  random_state=None, shrinking=True, tol=0.001,
                                  verbose=False, probability=True)
clf7=RandomForestClassifier(n_jobs=6,n_estimators=1000,max_depth=6,class_weight='balanced')
clf8=KNeighborsClassifier(n_neighbors=12)
clf9=DecisionTreeClassifier(class_weight='balanced')
neuronov_v_sloe=[]
for j in range(10):
    neuronov_v_sloe.append(100)
clf10=MLPClassifier(hidden_layer_sizes=neuronov_v_sloe)
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('NB', clf2), ('sgd', clf3),
                       ('xgb', clf4), ('svc', clf6), ('rf', clf7),
            ('knn', clf8), ('dt', clf9), ('perceptron', clf10)], voting='soft',weights=[1, 1, 1, 2, 1.3, 2, 1,1.3,1.5], n_jobs=6)

In [15]:
import joblib
from distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58141,Cluster Workers: 3 Cores: 6 Memory: 17.11 GB


In [16]:
with joblib.parallel_backend("dask"):
    eclf1.fit(x1, y1)

y_2=eclf1.predict(x2)
print(f1_score(y2,y_2))

0.9693251533742332


In [22]:
with joblib.parallel_backend("dask"):
    eclf1.fit(X_train_, y_train)

y_pred=eclf1.predict(X_test_)

In [ ]:
file = open("my_submission.csv", "wb")
file.write( bytes(str("Id,Prediction\n"), "utf-8"))
for i in range(len(test_docs)):
    file.write(bytes(str(test_docs[i][0]) + "," +str(int(y_pred[i])) + str("\n"), "utf-8"))
file.close()

#### пробовал doc2vec и смотрел косинувское расстояние между документами, но не очень получилось

In [ ]:
max_epochs = 100
alpha = 0.025

model = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0)

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i,doc in enumerate(generator(train_docss=train_docs+test_docs))]

In [ ]:
model.build_vocab(tagged_data)

In [ ]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

In [ ]:
coss=[]
cos=[]
for i in range(len(train_docs)):
    cos.clear()
    for j in range(len(train_docs)):
        similar_doc = model.docvecs.distance(i,j)
        cos.append(similar_doc)
    coss.append(cos)
    sys.stdout.write('\r {0}...'.format(i))
coss=np.matrix(coss)

In [ ]:
coss_test=[]
cos_test=[]
for i in range(len(train_docs),len(test_docs+train_docs)):
    cos_test.clear()
    for j in range(len(train_docs)):
        similar_doc = model.docvecs.distance(i,j)
        cos_test.append(similar_doc)
    coss_test.append(cos)
    sys.stdout.write('\r {0}...'.format(i))
coss_test=np.matrix(coss_test)

In [ ]:
np.save('coss_train.npy', coss)
np.save('coss_test.npy', coss_test)

In [ ]:
x_vs=sparse.csc_matrix(f1).tocsr()
x_cos_train=sparse.csc_matrix(coss).tocsr()
X_train_=sparse.hstack([X_train, x_vs]).tocsr()
x_vs=sparse.csc_matrix(f2).tocsr()
x_cos_test=sparse.csc_matrix(coss_test).tocsr()
X_test_=sparse.hstack([X_test, x_vs]).tocsr()

In [ ]:
model = xgboost.XGBClassifier(max_depth=6,learning_rate=0.3,n_estimators=1000,scale_pos_weight=1.5, n_jobs=6)

In [ ]:
model.fit(X_train_,y_train)
y_pred=model.predict(X_test_)

In [ ]:
file = open("my_submission.csv", "wb")
file.write( bytes(str("Id,Prediction\n"), "utf-8"))
for i in range(len(test_docs)):
    file.write(bytes(str(test_docs[i][0]) + "," +str(int(y_pred[i])) + str("\n"), "utf-8"))
file.close()